***GENERATED CODE FOR bostonhousingdatapipeline PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run bostonhousingdatapipelineHooks.ipynb
try:
	#sourcePreExecutionHook()

	bostontrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/Data/TrainData/BostonTrain.csv', 'filename': 'BostonTrain.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line'}")
	#sourcePostExecutionHook(bostontrain)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run bostonhousingdatapipelineHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(bostontrain,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "CRIM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "2.33", "stddev": "5.05", "min": "0.01432", "max": "22.5971", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CRIM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ZN", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "8.3", "stddev": "21.95", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ZN"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "INDUS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "9.55", "stddev": "5.79", "min": "1.32", "max": "19.58", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "INDUS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CHAS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "0.04", "stddev": "0.2", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CHAS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NOX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "0.54", "stddev": "0.1", "min": "0.411", "max": "0.74", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "NOX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "6.48", "stddev": "0.77", "min": "5.0", "max": "8.725", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AGE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "72.41", "stddev": "24.8", "min": "17.2", "max": "98.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "AGE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DIS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "3.76", "stddev": "2.09", "min": "1.5184", "max": "8.3248", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DIS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RAD", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "8.96", "stddev": "7.85", "min": "1.0", "max": "24.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RAD"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TAX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "399.52", "stddev": "149.09", "min": "193.0", "max": "666.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TAX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PTRATIO", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "18.38", "stddev": "1.99", "min": "14.7", "max": "21.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PTRATIO"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "B", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "340.71", "stddev": "114.3", "min": "0.32", "max": "396.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "B"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LSTAT", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "11.52", "stddev": "6.86", "min": "2.98", "max": "31.99", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LSTAT"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MEDV", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "22.64", "stddev": "10.65", "min": "7.4", "max": "50.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MEDV"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run bostonhousingdatapipelineHooks.ipynb
try:
	#sinkPreExecutionHook()

	classification = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/602b6352ad86763318df9d53/part-00000-d279bf9c-5d40-4140-a6d9-da475f1d6398-c000.csv', 'samplecount': 506, 'originalcount': 506, 'orignalKey': '/FileStore/platform/Data/TrainData/BostonTrain.csv', 'pathOnly': '/Data pipelines/Classification', 'project_id': '6176938b06f82d60f3a21add', 'parent_id': '6176938b06f82d60f3a21add', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '602b639bad86763318df9d73', 'field': 'CRIM', 'alias': 'CRIM', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '602b639bad86763318df9d74', 'field': 'ZN', 'alias': 'ZN', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '602b639bad86763318df9d75', 'field': 'INDUS', 'alias': 'INDUS', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '602b639bad86763318df9d76', 'field': 'CHAS', 'alias': 'CHAS', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '602b639bad86763318df9d77', 'field': 'NOX', 'alias': 'NOX', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '602b639bad86763318df9d78', 'field': 'RM', 'alias': 'RM', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '602b639bad86763318df9d79', 'field': 'AGE', 'alias': 'AGE', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '602b639bad86763318df9d7a', 'field': 'DIS', 'alias': 'DIS', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '602b639bad86763318df9d7b', 'field': 'RAD', 'alias': 'RAD', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '602b639bad86763318df9d7c', 'field': 'TAX', 'alias': 'TAX', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '602b639bad86763318df9d7d', 'field': 'PTRATIO', 'alias': 'PTRATIO', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '602b639bad86763318df9d7e', 'field': 'B', 'alias': 'B', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '602b639bad86763318df9d7f', 'field': 'LSTAT', 'alias': 'LSTAT', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '602b639bad86763318df9d80', 'field': 'MEDV', 'alias': 'MEDV', 'type': 'real'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c84d0ccbcb8b6f09e42113', 'field': 'CRIM', 'alias': 'CRIM', 'type': 'real', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c84d0ccbcb8b6f09e42114', 'field': 'ZN', 'alias': 'ZN', 'type': 'real', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c84d0ccbcb8b6f09e42115', 'field': 'INDUS', 'alias': 'INDUS', 'type': 'real', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c84d0ccbcb8b6f09e42116', 'field': 'CHAS', 'alias': 'CHAS', 'type': 'real', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c84d0ccbcb8b6f09e42117', 'field': 'NOX', 'alias': 'NOX', 'type': 'real', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c84d0ccbcb8b6f09e42118', 'field': 'RM', 'alias': 'RM', 'type': 'real', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c84d0ccbcb8b6f09e42119', 'field': 'AGE', 'alias': 'AGE', 'type': 'real', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c84d0ccbcb8b6f09e4211a', 'field': 'DIS', 'alias': 'DIS', 'type': 'real', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c84d0ccbcb8b6f09e4211b', 'field': 'RAD', 'alias': 'RAD', 'type': 'real', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c84d0ccbcb8b6f09e4211c', 'field': 'TAX', 'alias': 'TAX', 'type': 'real', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c84d0ccbcb8b6f09e4211d', 'field': 'PTRATIO', 'alias': 'PTRATIO', 'type': 'real', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c84d0ccbcb8b6f09e4211e', 'field': 'B', 'alias': 'B', 'type': 'real', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c84d0ccbcb8b6f09e4211f', 'field': 'LSTAT', 'alias': 'LSTAT', 'type': 'real', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c84d0ccbcb8b6f09e42120', 'field': 'MEDV', 'alias': 'MEDV', 'type': 'real', 'position': '13'}], 'server': 'https://ai.numtra.com', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'DataCatalogResults.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-4e492a83-2f57-4a64-926d-98bdd3bdeaed-c000.csv', 'results_url': 'http://3.140.115.35:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
